In [10]:
import torch.nn as nn 
import torch.nn.functional as F
import torch

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:40% !important; }</style>"))

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.ModuleList()
        
        self.layers+=[nn.Conv2d(3, 16,  kernel_size=3) , 
                      nn.ReLU(inplace=True)]
        self.layers+=[nn.Conv2d(16, 16,  kernel_size=3, stride=2), 
                      nn.ReLU(inplace=True)]
        self.layers+=[nn.Conv2d(16, 32,  kernel_size=3), 
                      nn.ReLU(inplace=True)]
        self.layers+=[nn.Conv2d(32, 32,  kernel_size=3, stride=2), 
                      nn.ReLU(inplace=True)]
        self.fc = nn.Linear(32*5*5, 10)
        
        
    def forward(self, x):
        for i in range(len(self.layers)):
            x = self.layers[i](x)
        x = x.view(-1, 32*5*5)
        x = self.fc(x)
        return x



In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch

def train(model, device, train_loader, optimizer, epoch, display=True):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
    if display:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [51]:
def getValSets(num,random_permute,cifar_data,cifar_data_val):
    sets = []
    for x in range(num):
        s = 200*x + 10
        indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[s:s+200]] for classe in range(0, 10)])
        subset = Subset(cifar_data_val, indx_val)
        sets.append(torch.utils.data.DataLoader(subset,
                                           batch_size=128, 
                                           shuffle=False))
    return sets

def getLatexRow(seed,net,acc,epoch,lr,dataAug="Nothing"):
    categories = ["seed","network","Accuracy","Epochs","learning rate","data augmentation"]
    row = [str(seed),str(net),str(round(acc,3)),str(epoch),str(lr),str(dataAug)]
    
    c = "&".join(categories)
    r = "&".join(row)
    return "{}\\\\\n{}\\\\\n".format(c,r)
    

In [52]:
from numpy.random import RandomState
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import Subset
import PIL
import torchvision

from torchvision import datasets, transforms

AUGMENT = False


normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])




if AUGMENT:
    dataAugmentation = [ 
        torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomRotation(20, resample=PIL.Image.BILINEAR)
    ]
    augment = "Color Jitter+HorizFlip+rotation"
else: 
    dataAugmentation = []
    augment = "Nothing"


transform_val = transforms.Compose([transforms.ToTensor(), normalize]) #careful to keep this one same
transform_train = transforms.Compose([transforms.ToTensor(), normalize] + dataAugmentation) 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

##### Cifar Data
cifar_data = datasets.CIFAR10(root='.',train=True, transform=transform_train, download=True)
    
#We need two copies of this due to weird dataset api 
cifar_data_val = datasets.CIFAR10(root='.',train=True, transform=transform_val, download=True)
    
# Extract a subset of 100 (class balanced) samples per class

accs = []

trainTRSF = torchvision.transforms.Compose([
torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
torchvision.transforms.RandomHorizontalFlip(),
torchvision.transforms.RandomRotation(20, resample=PIL.Image.BILINEAR)])

for seed in [1,2]:
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 5000))
    
    indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:10]] for classe in range(0, 10)])
    valSets = getValSets(10,random_permute,cifar_data,cifar_data_val)

    train_data = Subset(cifar_data, indx_train)
#     val_data = Subset(cifar_data_val, indx_val)
    

    print('Num Samples For Training {} Num Samples For Val {}'.format(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data,
                                             batch_size=128, 
                                             shuffle=True)

    val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=128, 
                                           shuffle=False)
    
    

    model = Net()
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), 
                              lr=0.01, momentum=0.9,
                              weight_decay=0.0005)
    for epoch in range(100):
        train(model, device, train_loader, optimizer, epoch, display=False)
        
    for val_loader in valSets:
        accs.append(test(model, device, val_loader))
        
    temp = getLatexRow(seed,
                        net="default",
                       acc=round(float(np.mean(accs)),3),
                       epoch=100,
                       lr=0.01,
                       dataAug=augment)
    print(temp)

accs = np.array(accs)
print('Acc over 2 instances: %.2f +- %.2f'%(accs.mean(),accs.std()))

Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 100 Num Samples For Val 2000

Test set: Average loss: 9.9919, Accuracy: 351/2000 (17.55%)


Test set: Average loss: 9.9847, Accuracy: 339/2000 (16.95%)


Test set: Average loss: 9.7995, Accuracy: 378/2000 (18.90%)


Test set: Average loss: 10.1040, Accuracy: 333/2000 (16.65%)


Test set: Average loss: 10.4743, Accuracy: 318/2000 (15.90%)


Test set: Average loss: 10.3609, Accuracy: 336/2000 (16.80%)


Test set: Average loss: 10.0508, Accuracy: 337/2000 (16.85%)


Test set: Average loss: 10.0247, Accuracy: 296/2000 (14.80%)


Test set: Average loss: 9.8532, Accuracy: 373/2000 (18.65%)


Test set: Average loss: 10.1306, Accuracy: 333/2000 (16.65%)

seed&network&Accuracy&Epochs&learning rate&data augmentation\\
1&default&16.97&100&0.01&Nothing\\

Num Samples For Training 100 Num Samples For Val 2000

Test set: Average loss: 3.1144, Accuracy: 343/2000 (17.15%)


Test set: Average loss: 3.09